In [ ]:
# part1
from pyspark import SparkConf,SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import*
from pyspark.sql.functions import*
from pyspark import SparkConf,SparkContext

In [ ]:
#sc.stop()
sc = SparkContext("local", "Demo App")
spark = SQLContext(sc)
spark_conf = SparkConf().setMaster('local').setAppName('DEMO')
spark._jvm.net.snowflake.spark.snowflake.SnowflakeConnectorUtils.enablePushdownSession(spark._jvm.org.apache.spark.sql.SparkSession.builder().getOrCreate())
sc._jvm.net.snowflake.spark.snowflake.SnowflakeConnectorUtils.enablePushdownSession(sc._jvm.org.apache.spark.sql.SparkSession.builder().getOrCreate())


In [ ]:
import getpass

# Enter the values of account, user, password, warehouse and role to match your environment
ACCOUNT = input('Snowflake Account Name: ')
USER =  input('Snowflake User Name: ')
PASSWORD = getpass.getpass('Snowflake Password: ')
WAREHOUSE = input('Snowflake Warehouse Name: ')
ROLE = input('Snowflake Role Name: ')

sfOptions={'sfURL': ACCOUNT+'.snowflakecomputing.com',
           'sfUser':USER,
           'sfPassword': PASSWORD,
           'sfDatabase':'SNOWFLAKE_SAMPLE_DATA',
           'sfSchema':'TPCH_SF1',
           'sfRole':ROLE,
           'sfWarehouse': WAREHOUSE}

sfSource='net.snowflake.spark.snowflake'

In [ ]:
df_nation=spark.read.format(sfSource) \
    .options(**sfOptions) \
    .option("dbtable","nation") \
    .load()

df_region=spark.read.format(sfSource) \
    .options(**sfOptions) \
    .option("dbtable","region") \
    .load()

df_cust=spark.read.format(sfSource) \
    .options(**sfOptions) \
    .option("dbtable","customer") \
    .load()


In [ ]:
df_loc = df_nation.join(df_region, df_nation['N_REGIONKEY'] == df_region['R_REGIONKEY']) 


In [ ]:
df_cl = df_loc.join(df_cust, df_loc['N_NATIONKEY'] == df_cust['C_NATIONKEY']) \
         .filter(col('R_NAME') == 'AFRICA') \
         .select('C_MKTSEGMENT') \
         .groupBy('C_MKTSEGMENT').count()


In [ ]:
%%time
df_cl.show()

In [ ]:
df_cl.printSchema()

In [ ]:

df_cl_single = df_loc.join(df_cust, df_loc['N_NATIONKEY'] == df_cust['C_NATIONKEY']) \
         .filter(df_cust['C_CUSTKEY'] == '123456') \
         .select('C_MKTSEGMENT') \
         .groupBy('C_MKTSEGMENT').count()


In [ ]:
%%time
df_cl_single.show()